In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import CRPS.CRPS as pscore


pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.dumas_evaluation import *

In [2]:
with open('../../../models/artificial/dumas_model.pickle', 'rb') as f:
    dumas_model = pickle.load(f)

with open('../../../models/artificial/dumas_model_no_resource.pickle', 'rb') as f:
    dumas_model_no_resource = pickle.load(f)

In [3]:
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

N = 1000

In [4]:
with open('../transformed_event_logs/artificial_start_end_2_train.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [5]:
evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                 },
                                     train_data, n=N)
likelihoods_train_A_R = evaluator.sample_cases(plot_cases=False, multiprocessing=False)

100%|██████████████████████████████████████████████| 1802/1802 [08:50<00:00,  3.39it/s]


In [6]:
np.mean([v.ln() for v in likelihoods_train_A_R[0].values()])

Decimal('-11.09266236220206472456143419')

In [7]:
np.mean(get_pscores(likelihoods_train_A_R))

np.float64(9620.070722843333)

In [8]:
with open('../transformed_event_logs/artificial_start_end_2_test.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [9]:
evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                 },
                                     test_data, n=N)
likelihoods_test_A_R = evaluator.sample_cases(plot_cases=False, multiprocessing=False)

100%|████████████████████████████████████████████████| 892/892 [04:25<00:00,  3.36it/s]


In [10]:
np.mean([v.ln() for v in likelihoods_test_A_R[0].values()])

Decimal('-11.54052512055734716436593603')

In [11]:
np.mean(get_pscores(likelihoods_test_A_R))

np.float64(9855.46452988503)

In [12]:
evaluator = conduct_evaluation.ConductEvaluation(dumas_model_no_resource, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                 },
                                     train_data, n=N)
likelihoods_train_A = evaluator.sample_cases(plot_cases=False, multiprocessing=False)

100%|██████████████████████████████████████████████| 1802/1802 [08:51<00:00,  3.39it/s]


In [13]:
np.mean([v.ln() for v in likelihoods_train_A[0].values()])

Decimal('-11.20349764977855420773187591')

In [14]:
np.mean(get_pscores(likelihoods_train_A))

np.float64(9887.346417640118)

In [15]:
evaluator = conduct_evaluation.ConductEvaluation(dumas_model_no_resource, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource',
                                                 },
                                     test_data, n=N)
likelihoods_test_A = evaluator.sample_cases(plot_cases=False, multiprocessing=False)

100%|████████████████████████████████████████████████| 892/892 [04:23<00:00,  3.39it/s]


In [16]:
np.mean([v.ln() for v in likelihoods_test_A[0].values()])

Decimal('-13.01212982265674538014732453')

In [17]:
np.mean(get_pscores(likelihoods_test_A))

np.float64(10135.955341589937)

In [19]:
results = {
    'dumas_test_A' : likelihoods_test_A,
    'dumas_test_A_R' : likelihoods_test_A_R,
    'dumas_train_A' : likelihoods_train_A,
    'dumas_train_A_R' : likelihoods_train_A_R
}

with open('./artificial_dumas_evaluation_.pickle', 'wb') as handle:
    pickle.dump(results, handle)